## Serialization

Now we have a generic language for contracts, the first thing we might want to do is figure out how to serialize them. Once we have a serialization mechanism, we can transmit contracts across the wire, store them in databases and so on.

Personally I tend to reach for JSON whenever possible, because there's no point in re-inventing the wheel.

So if we can turn any contract into a jsonable data structure, we get serialization almost for free.

It turns out this is pretty easy. Contracts are already trees, and trees can be seen as nested lists:

```
[
    "foo",
    [
        "bar",
        [
            "and so on..."
        ]
    ]
]
```


So our serialization/deserialization strategy can be seen as:

`Contract -> List -> JSON -> List -> Contract`


In [1]:
from decimal import Decimal 
import binascii
from contracts import *

In [2]:
def spottrade(left_amount:Decimal, left:Asset, right_amount:Decimal, right:Asset):
    return AND(
        SCALE(CONST(left_amount),ONE(left)),
        GIVE(SCALE(CONST(right_amount),ONE(right)))
    )

In [3]:

t1=spottrade(1,BTC,20000,USD)

In [4]:
t1

AND(SCALE(CONST(1),ONE(BTC)),GIVE(SCALE(CONST(20000),ONE(USD))))

In [5]:
to_list(t1)

['AND',
 ['SCALE', ['CONST', 1], ['ONE', 'BTC']],
 ['GIVE', ['SCALE', ['CONST', 20000], ['ONE', 'USD']]]]

In [6]:
from_list(to_list(t1))

AND(SCALE(CONST(1),ONE(BTC)),GIVE(SCALE(CONST(20000),ONE(USD))))

In [7]:
dumps(t1)

'["AND", ["SCALE", ["CONST", 1], ["ONE", "BTC"]], ["GIVE", ["SCALE", ["CONST", 20000], ["ONE", "USD"]]]]'

In [8]:
loads(dumps(t1))

AND(SCALE(CONST(1),ONE(BTC)),GIVE(SCALE(CONST(20000),ONE(USD))))

In [9]:
loads(dumps(t1))==t1

True

We can even get bytes for a wire representation if we want:

In [10]:

b=binascii.hexlify(dumps(t1).encode())
b

b'5b22414e44222c205b225343414c45222c205b22434f4e5354222c20315d2c205b224f4e45222c2022425443225d5d2c205b2247495645222c205b225343414c45222c205b22434f4e5354222c2032303030305d2c205b224f4e45222c2022555344225d5d5d5d'

In [11]:
loads(binascii.unhexlify(b).decode())

AND(SCALE(CONST(1),ONE(BTC)),GIVE(SCALE(CONST(20000),ONE(USD))))